In [256]:
%config ZMQInteractiveShell.ast_node_interactivity='all'
%matplotlib inline
import warnings;warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [134]:
target = '标签'
uid = '申请编号'
columns = {}
columns['no_fea'] = [uid, target, '申请时点']
columns['01'] = [
    '贷款类型',
    '是否提供手机号',
    '是否提供电话',
    '手机号是否有效',
    '是否提供email',
    '申请人是否额外提供了文件2',
    '申请人是否额外提供了文件3',
    '申请人是否额外提供了文件4',
    '申请人是否额外提供了文件5',
    '申请人是否额外提供了文件6',
    '申请人是否额外提供了文件7',
    '申请人是否额外提供了文件8',
    '申请人是否额外提供了文件9',
    '是否有车',
    '是否有房',
    '地址是否一致标志1',
    '地址是否一致标志2',
    '地址是否一致标志3',
    '地址是否一致标志4',
    '地址是否一致标志5',
    '地址是否一致标志6']
columns['100'] = [
    '性别',
    '客户居住地评分1',
    '客户居住地评分2',
    '教育程度',
    '婚姻状态',
    '居住状态',
    '贷款申请前1小时内征信查询次数',
    '贷款申请前1天内征信查询次数',
    '贷款申请前1周内征信查询次数',
    '贷款申请前1个月内征信查询次数',
    '贷款申请前1个季度内征信查询次数',
    '贷款申请前1年内征信查询次数',
    '收入类型',
    '陪同申请人',
    '孩子个数',
    '职业',
    '家庭成员数',
    '社交圈违约信息1_1',
    '社交圈违约信息2_1',
    '有车时间',
    '单位类型']
columns['time'] = [
    '申请周内日',
    '申请时点',
    '出生日期距申请日期天数']
columns['_dt'] = [
    '最近一次换手机号码距申请日天数',
    '出生日期距申请日期天数',
    '工作日期距申请日期天数',
    '注册日期距申请日期天数',
    '身份认证日期距申请日期天数']
columns['money'] = [
    '贷款年金',
    '客户收入',
    '商品价格',
    '信用额度']

In [12]:
############## Functions

def ana_dtypes(t):
    if t == 'str':
        cols = types[types == 'object'].index
    else:
        cols = types[types != 'object'].index
    # Save description
    cols_desc = d[cols].describe().T
    cols_desc.to_csv(f'./data/cols_desc_{t}.csv')
    # Return columns
    return cols

a = d11.select_dtypes(include=['int64'])

In [13]:
############## Load

d0 = pd.read_csv('./data/train_label.csv')
d0.shape
d0.head()

# Target Count
d0['标签'].value_counts()

(140000, 2)

,申请编号,标签
0,0,1
1,1,0
2,2,0
3,3,0
4,5,0


0    119000
1     21000
Name: 标签, dtype: int64

In [17]:
d1 = pd.read_csv('./data/train_Application.csv')
d1.shape
d1.head()
d1.dtypes.value_counts()

(140000, 31)

,申请编号,贷款类型,信用额度,贷款年金,商品价格,陪同申请人,出生日期距申请日期天数,工作日期距申请日期天数,注册日期距申请日期天数,身份认证日期距申请日期天数,...,申请人是否额外提供了文件6,申请人是否额外提供了文件7,申请人是否额外提供了文件8,申请人是否额外提供了文件9,贷款申请前1小时内征信查询次数,贷款申请前1天内征信查询次数,贷款申请前1周内征信查询次数,贷款申请前1个月内征信查询次数,贷款申请前1个季度内征信查询次数,贷款申请前1年内征信查询次数
0,0,0,460190.889355,17463.042019,419951.511045,7.0,-10107,-342,-5421,-3292,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,424370.659603,15585.046388,384597.975692,7.0,-13980,-1110,-3387,-826,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
2,2,0,469330.587153,18837.558252,419951.511045,1.0,-13331,-2246,-3870,-171,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
3,3,0,464188.353140,16754.587069,409850.500944,7.0,-16540,292204,-970,-2916,...,0,0,0,1,0.0,0.0,0.0,0.0,0.0,1.0
4,5,0,434196.988738,16165.009200,394698.985793,7.0,-17919,-11037,-9350,-3588,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,5.0


int64      20
float64    11
dtype: int64

In [20]:
d2 = pd.read_csv('./data/train_Personas.csv')
d2.shape
d2.dtypes.value_counts()
d2.head()

(140000, 32)

,申请编号,性别,是否有车,是否有房,孩子个数,客户收入,收入类型,教育程度,婚姻状态,居住状态,...,地址是否一致标志3,地址是否一致标志4,地址是否一致标志5,地址是否一致标志6,单位类型,社交圈违约信息2_2,社交圈违约信息2_1,社交圈违约信息1_2,社交圈违约信息1_1,最近一次换手机号码距申请日天数
0,0,1,0,0,0,237349.580536,1,2,3,1,...,0,0,0,0,5,0.0,0.0,0.0,0.0,-887.0
1,1,1,0,1,0,237236.363407,7,4,1,1,...,0,0,0,1,5,0.0,0.0,0.0,0.0,-271.0
2,2,0,1,1,-1,237321.276253,1,4,1,1,...,1,0,0,0,5,0.0,0.0,0.0,0.0,-332.0
3,3,0,0,1,-1,237304.293684,3,4,2,1,...,0,0,0,0,57,1.0,0.0,1.0,0.0,-204.0
4,5,0,0,0,-1,237321.276253,1,1,2,1,...,1,1,0,1,53,0.0,0.0,0.0,0.0,-1049.0


int64      18
float64    14
dtype: int64

In [231]:
### Merge Application

d11 = pd.merge(d0, d1, how='inner', on='申请编号')
d11.shape
d11.head()

d11 = pd.merge(d11, d2, how='inner', on='申请编号')
d11.shape
d11.head()

d11.to_csv(f'./tmp/0_merge11.csv')

(140000, 32)

,申请编号,标签,贷款类型,信用额度,贷款年金,商品价格,陪同申请人,出生日期距申请日期天数,工作日期距申请日期天数,注册日期距申请日期天数,...,申请人是否额外提供了文件6,申请人是否额外提供了文件7,申请人是否额外提供了文件8,申请人是否额外提供了文件9,贷款申请前1小时内征信查询次数,贷款申请前1天内征信查询次数,贷款申请前1周内征信查询次数,贷款申请前1个月内征信查询次数,贷款申请前1个季度内征信查询次数,贷款申请前1年内征信查询次数
0,0,1,0,460190.889355,17463.042019,419951.511045,7.0,-10107,-342,-5421,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,0,424370.659603,15585.046388,384597.975692,7.0,-13980,-1110,-3387,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
2,2,0,0,469330.587153,18837.558252,419951.511045,1.0,-13331,-2246,-3870,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
3,3,0,0,464188.353140,16754.587069,409850.500944,7.0,-16540,292204,-970,...,0,0,0,1,0.0,0.0,0.0,0.0,0.0,1.0
4,5,0,0,434196.988738,16165.009200,394698.985793,7.0,-17919,-11037,-9350,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,5.0


(140000, 63)

,申请编号,标签,贷款类型,信用额度,贷款年金,商品价格,陪同申请人,出生日期距申请日期天数,工作日期距申请日期天数,注册日期距申请日期天数,...,地址是否一致标志3,地址是否一致标志4,地址是否一致标志5,地址是否一致标志6,单位类型,社交圈违约信息2_2,社交圈违约信息2_1,社交圈违约信息1_2,社交圈违约信息1_1,最近一次换手机号码距申请日天数
0,0,1,0,460190.889355,17463.042019,419951.511045,7.0,-10107,-342,-5421,...,0,0,0,0,5,0.0,0.0,0.0,0.0,-887.0
1,1,0,0,424370.659603,15585.046388,384597.975692,7.0,-13980,-1110,-3387,...,0,0,0,1,5,0.0,0.0,0.0,0.0,-271.0
2,2,0,0,469330.587153,18837.558252,419951.511045,1.0,-13331,-2246,-3870,...,1,0,0,0,5,0.0,0.0,0.0,0.0,-332.0
3,3,0,0,464188.353140,16754.587069,409850.500944,7.0,-16540,292204,-970,...,0,0,0,0,57,1.0,0.0,1.0,0.0,-204.0
4,5,0,0,434196.988738,16165.009200,394698.985793,7.0,-17919,-11037,-9350,...,1,1,0,1,53,0.0,0.0,0.0,0.0,-1049.0


In [215]:
''' 重复？'''
d11[uid].nunique()

''' Target '''
d11[target].value_counts()

''' Dtypes '''
d11.dtypes.value_counts()

''' Description '''
d_feats = d11.drop([uid, target], axis=1)
cols_desc = d_feats.describe().T
# Unique
cols_unique = d_feats.apply(pd.Series.nunique, axis = 0)
cols_unique = pd.DataFrame(cols_unique, columns=['unique'])
cols_desc = cols_desc.join(cols_unique)

cols_desc.head()
cols_desc.to_csv(f'./tmp/0_cols_desc.csv')

' 重复？'

140000

' Target '

0    119000
1     21000
Name: 标签, dtype: int64

' Dtypes '

int64      33
float64    30
dtype: int64

' Description '

,count,mean,std,min,25%,50%,75%,max,unique
贷款类型,140000.0,0.092671,0.289972,0.000000,0.000000,0.000000,0.000000,1.000000,2
信用额度,140000.0,478907.188397,51221.158226,408260.788232,437110.844412,468139.656834,506177.878906,921791.788232,4516
贷款年金,139993.0,17123.620552,1401.602040,14650.637578,16103.385420,16914.983717,17852.888912,36847.435823,11666
商品价格,139880.0,429468.479262,41142.193912,374496.965591,396214.137308,419951.511045,445709.086803,823991.915085,714
陪同申请人,139430.0,6.082084,2.116810,0.000000,7.000000,7.000000,7.000000,7.000000,7


In [67]:
############## Outlier：Get

low = .01
high = .99
quant_df = d11.drop([uid, target], axis=1).quantile([low, high]).T
quant_df.to_csv('./tmp/0_outlier.csv')

In [241]:
# 处理
d12 = d11.copy()

# 置空值
d12.loc[d12['孩子个数']==-1, '孩子个数'] = np.NaN
# 距申请日期>0
for col in columns['_dt']:
    d12.loc[d12[col]>0, col] = np.NaN

In [242]:
############## Outlier：Apply

outliers = pd.read_csv('./tmp/0_outlier1.csv', index_col=0)
outliers.shape
outliers.head()

def set_outlier(col):
    if col < col_min:
        col = col_min
    elif col > col_max:
        col = col_max
    return col

# def set_outlier(col):
#     if col < col_min or col > col_max:
#         col = np.NaN
#     return col
    
for col in outliers.index:
# for col in '出生日期距申请日期天数', '社交圈违约信息1_2':
    col_min = outliers.loc[col, 'min']
    col_max = outliers.loc[col, 'max']
    d12[col] = d12[col].apply(set_outlier)
    
d12.to_csv(f'./tmp/2_eda_outlier.csv')

(61, 2)

,min,max
地址是否一致标志1,0,1
地址是否一致标志2,0,1
地址是否一致标志3,0,1
地址是否一致标志4,0,1
地址是否一致标志5,0,1


In [255]:
############# correlations

correlations = d12.drop([uid], axis=1).corr()
correlations.to_csv('./tmp/0_correlations.csv')

In [225]:
d12.loc[d12['出生日期距申请日期天数'], '身份认证日期距申请日期天数'].min()

nan

In [125]:
for col in ['社交圈违约信息1_2']:
    d12[col].value_counts()

0.0     97388
1.0     13455
2.0      9171
3.0      6398
4.0      4279
6.0      3296
5.0      2906
7.0       887
8.0       589
9.0       361
10.0      284
11.0      183
13.0      125
12.0      102
14.0       37
20.0       24
16.0       23
18.0       16
15.0       13
17.0       13
19.0        7
Name: 社交圈违约信息1_2, dtype: int64

In [285]:
############## Feature Engineering

### Correlation

def calc_poly(cols):
    degree = 2
    poly_features = d12[cols]
    poly_target = d12[target]

    # Fill Miss
    from sklearn.preprocessing import Imputer
    imputer = Imputer(strategy = 'median')
    poly_features = imputer.fit_transform(poly_features)

    # Polynomial
    from sklearn.preprocessing import PolynomialFeatures
    poly_transformer = PolynomialFeatures(degree = degree, include_bias=False) # interaction_only
    poly_transformer.fit(poly_features)
    poly_features = poly_transformer.transform(poly_features)
    poly_features = pd.DataFrame(poly_features, 
                                 columns = poly_transformer.get_feature_names(cols))
    return poly_features


cols = ['出生日期距申请日期天数',
        '工作日期距申请日期天数',
        '最近一次换手机号码距申请日天数',
        '客户居住地评分2',
        '客户居住地评分1',
        '教育程度',
        '性别',
        '身份认证日期距申请日期天数',
        '地址是否一致标志6',
        '收入类型',
        '是否提供手机号',
        '地址是否一致标志2',
        '注册日期距申请日期天数',
        '有车时间']
#         '居住状态',
#         '地址是否一致标志4',
#         '是否提供电话',
#         '职业',
#         '社交圈违约信息2_1',
#         '贷款申请前1年内征信查询次数',
#         '社交圈违约信息1_1']
poly_features = calc_poly(cols)
poly_features.shape
poly_features.head()

(140000, 119)

,出生日期距申请日期天数,工作日期距申请日期天数,最近一次换手机号码距申请日天数,客户居住地评分2,客户居住地评分1,教育程度,性别,身份认证日期距申请日期天数,地址是否一致标志6,收入类型,...,是否提供手机号^2,是否提供手机号 地址是否一致标志2,是否提供手机号 注册日期距申请日期天数,是否提供手机号 有车时间,地址是否一致标志2^2,地址是否一致标志2 注册日期距申请日期天数,地址是否一致标志2 有车时间,注册日期距申请日期天数^2,注册日期距申请日期天数 有车时间,有车时间^2
0,-10107.0,-342.0,-887.0,0.0,0.0,2.0,1.0,-3292.0,0.0,1.0,...,1.0,0.0,-5421.0,4.0,0.0,-0.0,0.0,29387241.0,-21684.0,16.0
1,-13980.0,-1110.0,-271.0,1.0,1.0,4.0,1.0,-826.0,1.0,7.0,...,1.0,1.0,-3387.0,4.0,1.0,-3387.0,4.0,11471769.0,-13548.0,16.0
2,-13331.0,-2246.0,-332.0,0.0,0.0,4.0,0.0,-171.0,0.0,1.0,...,1.0,0.0,-3870.0,1.0,0.0,-0.0,0.0,14976900.0,-3870.0,1.0
3,-16540.0,-1278.0,-204.0,1.0,1.0,4.0,0.0,-2916.0,0.0,3.0,...,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,940900.0,-3880.0,16.0
4,-17919.0,-11037.0,-1049.0,0.0,0.0,1.0,0.0,-3588.0,1.0,1.0,...,1.0,0.0,-9350.0,4.0,0.0,-0.0,0.0,87422500.0,-37400.0,16.0


In [283]:
d13 = pd.concat([d12, poly_features.drop(cols, axis=1)], axis=1)
d13.shape
d13.head()

d13.to_csv(f'./tmp/3_poly.csv')

(140000, 168)

,申请编号,标签,贷款类型,信用额度,贷款年金,商品价格,陪同申请人,出生日期距申请日期天数,工作日期距申请日期天数,注册日期距申请日期天数,...,是否提供手机号^2,是否提供手机号 地址是否一致标志2,是否提供手机号 注册日期距申请日期天数,是否提供手机号 有车时间,地址是否一致标志2^2,地址是否一致标志2 注册日期距申请日期天数,地址是否一致标志2 有车时间,注册日期距申请日期天数^2,注册日期距申请日期天数 有车时间,有车时间^2
0,0,1,0,460190.889355,17463.042019,419951.511045,7.0,-10107.0,-342.0,-5421.0,...,1.0,0.0,-5421.0,4.0,0.0,-0.0,0.0,29387241.0,-21684.0,16.0
1,1,0,0,424370.659603,15585.046388,384597.975692,7.0,-13980.0,-1110.0,-3387.0,...,1.0,1.0,-3387.0,4.0,1.0,-3387.0,4.0,11471769.0,-13548.0,16.0
2,2,0,0,469330.587153,18837.558252,419951.511045,1.0,-13331.0,-2246.0,-3870.0,...,1.0,0.0,-3870.0,1.0,0.0,-0.0,0.0,14976900.0,-3870.0,1.0
3,3,0,0,464188.353140,16754.587069,409850.500944,7.0,-16540.0,NaN,-970.0,...,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,940900.0,-3880.0,16.0
4,5,0,0,434196.988738,16165.009200,394698.985793,7.0,-17919.0,-11037.0,-9350.0,...,1.0,0.0,-9350.0,4.0,0.0,-0.0,0.0,87422500.0,-37400.0,16.0


In [268]:
poly_transformer.get_feature_names(input_features = cols)

['1',
 '出生日期距申请日期天数',
 '工作日期距申请日期天数',
 '最近一次换手机号码距申请日天数',
 '出生日期距申请日期天数^2',
 '出生日期距申请日期天数 工作日期距申请日期天数',
 '出生日期距申请日期天数 最近一次换手机号码距申请日天数',
 '工作日期距申请日期天数^2',
 '工作日期距申请日期天数 最近一次换手机号码距申请日天数',
 '最近一次换手机号码距申请日天数^2',
 '出生日期距申请日期天数^3',
 '出生日期距申请日期天数^2 工作日期距申请日期天数',
 '出生日期距申请日期天数^2 最近一次换手机号码距申请日天数',
 '出生日期距申请日期天数 工作日期距申请日期天数^2',
 '出生日期距申请日期天数 工作日期距申请日期天数 最近一次换手机号码距申请日天数',
 '出生日期距申请日期天数 最近一次换手机号码距申请日天数^2',
 '工作日期距申请日期天数^3',
 '工作日期距申请日期天数^2 最近一次换手机号码距申请日天数',
 '工作日期距申请日期天数 最近一次换手机号码距申请日天数^2',
 '最近一次换手机号码距申请日天数^3']

In [22]:
d3 = pd.read_csv('./data/train_History_Application.csv')
d3.shape
d3.dtypes.value_counts()
d3.head()

(647963, 32)

int64      19
float64    11
object      2
dtype: int64

,历史申请编号,申请编号,贷款类型,贷款年金,申请额度,信用额度,首付金额,商品价格,申请周内日,申请时点,...,商品类别,组合类型,产品类型,获客渠道,销售区域,行业,贷款期数,收益级别,产品组合,首付时间
0,1040733,173562,0,4444.0,106931,119633.0,NaN,106931.0,5,5,...,27,2,2,3,0,10,44.0,2,7.0,292204.0
1,450011,173562,0,3831.0,79211,90188.0,NaN,79211.0,7,4,...,27,2,2,0,4,10,44.0,3,7.0,292204.0
2,698695,173562,A,861.0,8327,4180.0,4367.0,8327.0,5,5,...,2,3,0,7,84,4,7.0,1,10.0,292204.0
3,924081,173562,A,1183.0,12568,12568.0,11.0,12568.0,7,4,...,7,3,0,7,83,5,14.0,3,12.0,292204.0
4,907875,173562,1,1990.0,11,39611.0,NaN,NaN,6,6,...,27,0,2,3,0,10,0.0,0,1.0,-560.0


In [23]:
types = d3.dtypes
types[types == 'object']

贷款类型           object
是否为最后一次申请记录    object
dtype: object

In [24]:
for col in '贷款类型', '是否为最后一次申请记录':
    d3[col].value_counts()

0    287881
A    284292
1     75624
Name: 贷款类型, dtype: int64

Y    644478
N      3485
Name: 是否为最后一次申请记录, dtype: int64

In [32]:
d4 = pd.read_csv('./data/train_History_Payment.csv')
d4.shape
d4.dtypes.value_counts()
d4.head()

(5257909, 8)

int64      5
float64    3
dtype: int64

,历史申请编号,申请编号,分期付款日期,有支付行为的分期付款,分期付款应支付时间,分期付款实际支付时间,本期贷款金额,本期还款金额
0,225186,81363,2,8,-934,-939.0,1233.0,1233.0
1,877876,81363,2,26,-426,-431.0,2000.0,2000.0
2,225186,81363,2,3,-1030,-1035.0,1233.0,1233.0
3,518289,81363,2,3,-182,-184.0,1308.0,1308.0
4,877876,81363,2,13,-690,-695.0,2000.0,2000.0
